<a href="https://colab.research.google.com/github/Ghonem22/Arabic-Sentiment-Analysis-With-Emojis/blob/main/Arabic_Sentiment_Analysis_With_Emojis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libraries

In [1]:
# to use transformers
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/datasets.git
!pip install swifter
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-698li_3v
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-698li_3v
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.3 MB/s 
     |████████████████████████████████| 163 kB 55.9 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5526118 sha256=e467cd8215346d717c802a61c57e712203bd2c8a1ab38589cd6599ba377cb5aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-v753feg7/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Change Dir

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/UPwork/sentiment

/content/drive/MyDrive/UPwork/sentiment


## Import required libraries

In [4]:
import numpy as np
import pandas as pd
import re
import sys
import os
os.environ['KERAS_BACKEND']='tensorflow' 
import matplotlib.pyplot as plt
plt.switch_backend('agg')
get_ipython().run_line_magic('matplotlib', 'inline')
import string
import warnings
import yaml
import torch

warnings.filterwarnings("ignore")

with open(r'emojis.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    emojis = yaml.load(file, Loader=yaml.FullLoader)


## Read Data

In [5]:
# Read Text Files with Pandas using read_csv()

# importing pandas
import pandas as pd

# read text file into pandas DataFrame
df = pd.read_csv("1.txt", sep="\t")

# display DataFrame
df

,Tweet Text
0,@All_Qatar_2022 الوضع السلبي جدا والقادم اسوا ...
1,@1vnrc @LGBTQarabic ادوس عليك
2,ظاهرة اختطاف الأطفال المسلمين أنتشرت بالتزامن ...
3,@1lIolp @shibl_alshibl @LGBTQarabic طيب ستر اف...
4,الدول العربية ترفض تقبل #الشذوذ و #المثليين وه...
...,...
127792,الله يبدكم يا المثليين قولو امين يالمثليين
127793,ومع عامنا الجديد يااااررب استجب دعائي ان المثل...
127794,الله يلعنهم الله يلعنهم سخطونا ياجماعه المثليي...
127795,ترقُبون الانفاق.! ودائماً ارقام المال حاضرة في...


## Clean Data

In [6]:

def remove_special(text):
    for letter in '#.][!XR':
      text = text.replace(letter,'')
    return text

def remove_punctuations(text):
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
    
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text     
        
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)  

def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    # #remove longation
    # p_longation = re.compile(r'(.)\1+')
    # subst = r"\1\1"
    # text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

    
def keep_only_arabic(text):
    return re.sub(r'[a-zA-Z?]', '', text).strip()
    


def split_hashtag_to_words(tag):
    tag = tag.replace('#','')
    tags = tag.split('_')
    if len(tags) > 1 :
        
        return tags
    pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")
    return pattern.findall(tag)

def clean_hashtag(text):
    words = text.split()
    text = list()
    for word in words:
        if is_hashtag(word):
            text.extend(extract_hashtag(word))
        else:
            text.append(word)
    return " ".join(text)

def is_hashtag(word):
    if word.startswith("#"):
        return True
    else:
        return False

def extract_hashtag(text):
    
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list :
        word_list.extend(split_hashtag_to_words(word))
    return word_list

def remove_mentions(text):
    words = text.split()
    words = [w for w in words if not w.startswith("@")]
    return " ".join(words)

def preprocess_text(text): 
    
    text = remove_mentions(text)
    #Replace @username with empty string
    text = re.sub('@[^\s]+', ' ', text)

    #Convert www.* or https?://* to " "
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)

    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    # remove punctuations
    text= remove_punctuations(text)

    # normalize the text
    text= normalize_arabic(text)

    # remove repeated letters
    # text=remove_repeating_char(text)

    # remove special letters
    text=remove_special(text)

  # Clean/Normalize Arabic Text
    text = clean_str(text)

  # remove english words
    # text = keep_only_arabic(text)
  # stemming
    #text= stemmer.stem(text)
              
    return text


preprocess_text("@LGBTQarabic 🌈❤🖤🖤❤🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈")

'🌈❤🖤🖤❤🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈🌈'

In [7]:
df['clean_text']=df['Tweet Text'].apply(preprocess_text)
df.head(15)

,Tweet Text,clean_text
0,@All_Qatar_2022 الوضع السلبي جدا والقادم اسوا ...,الوضع السلبي جدا والقادم اسوا مع المثلين وقس ع...
1,@1vnrc @LGBTQarabic ادوس عليك,ادوس عليك
2,ظاهرة اختطاف الأطفال المسلمين أنتشرت بالتزامن ...,ظاهره اختطاف الاطفال المسلمين انتشرت بالتزامن ...
3,@1lIolp @shibl_alshibl @LGBTQarabic طيب ستر اف...,طيب ستر افتارك اول
4,الدول العربية ترفض تقبل #الشذوذ و #المثليين وه...,الدول العربيه ترفض تقبل الشذوذ والمثلين وهذا ا...
5,@historydefined هسه يجون المثليين يمتبون me an...,هسه يجون المثلين يمتبون me and who
6,@1lIolp @shibl_alshibl @LGBTQarabic محد سألك,محد سالك
7,@LGBTQarabic عقبال الأردن,عقبال الاردن
8,@LGBTQarabic Congrats,Congrats
9,اين جماعة الإخوان من كأس العالم الذي يقام في ق...,اين جماعه الاخوان من كاس العالم الذي يقام في ق...


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")


In [9]:
def emoje_score(sentence, logits):
    # check +ve emojes
    n = 0
    for pos_emoj in emojis['positive']:
        if pos_emoj in sentence:
            logits[0][0] += emojis['emoje_weight']
            n += 1

        if n > 5: 
            break
            
    # check -ve emojes
    n = 0
    for neg_emoj in emojis['negative']:
        if pos_emoj in sentence:
            logits[0][1] += emojis['emoje_weight']
            n += 1

        if n > 5: 
            break

    return logits

# this return prediction only
def Predict1(sentence):
    try:
        sentence = sentence[:1600]
        sentence = preprocess_text(sentence)
        labels = ["Positive", 'Negative', "Nutral"]
        inputs = tokenizer(sentence, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = emoje_score(sentence, logits)
        prediction = int(torch.argmax(logits, dim=-1))
        logits_np = logits.cpu().detach().numpy()
        prob = (np.exp(logits_np) / np.sum(np.exp(logits_np))) * 100
        return labels[prediction]
    except:
        return "Nutral"


# Prediction with confidence

def Predict2(sentence):
    try:
        sentence = sentence[:1600]
        sentence = preprocess_text(sentence)
        labels = ["Positive", 'Negative', "Nutral"]
        inputs = tokenizer(sentence, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = emoje_score(sentence, logits)
        prediction = int(torch.argmax(logits, dim=-1))
        logits_np = logits.cpu().detach().numpy()
        prob = (np.exp(logits_np) / np.sum(np.exp(logits_np))) * 100
        return labels[prediction], prob[0][prediction]
    except:
        return "Nutral" , 0.5

## Apply model to dataframe

In [10]:
import time

# Take sample of the data
df_sample = df.iloc[:1000]
t = time.time()
from tqdm import tqdm
tqdm.pandas()

# if you want result with confidence, replace Predict1 with Predict2
df_sample['predictions'] = df_sample['clean_text'].progress_apply(lambda x: Predict1(x))

# save result as csv
df_sample.to_csv('result_sample.csv')

time.time() - t

100%|██████████| 1000/1000 [03:49<00:00,  4.35it/s]


230.0339114665985

In [12]:
df_sample.tail()

,Tweet Text,clean_text,predictions
995,@BBCArabic هذا المخنث احضر المطبوعات معه من خا...,هذا المخنث احضر المطبوعات معه من خارج البلاد ع...,Negative
996,قريتها المثليين انحولت من الدراسة,قريتها المثلين انحولت من الدراسه,Negative
997,@t_donkeys اولاً اسمهم المخن*ثين وليس المثليين,اولا اسمهم المخنثين وليس المثلين,Negative
998,@QGU_5 يراد بهالوكت الي المثليين صاروا مظلومين...,يراد بهالوكت الي المثلين صاروا مظلومين الله يك...,Negative
999,#مجتمع_الميم هذا المجتمع القذر مجتمع اللواط وا...,مجتمعالميم هذا المجتمع القذر مجتمع اللواط والس...,Negative


## Apply to all data

**THis take a lot of time, maybe 4 hours**

In [ ]:
# import time

# # Take sample of the data
# t = time.time()
# from tqdm import tqdm
# tqdm.pandas()

# df['predictions'] = df['clean_text'].progress_apply(lambda x: Predict1(x))
# df.to_csv('result_1.csv')

# time.time() - t

## Apply the model to one sentnece

In [13]:
# Prediction with confidence

def Predict1(sentence):
    try:
        sentence = sentence[:1600]
        sentence = preprocess_text(sentence)
        labels = ["Positive", 'Negative', "Nutral"]
        inputs = tokenizer(sentence, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = emoje_score(sentence, logits)
        prediction = int(torch.argmax(logits, dim=-1))
        logits_np = logits.cpu().detach().numpy()
        prob = (np.exp(logits_np) / np.sum(np.exp(logits_np))) * 100
        return labels[prediction], prob[0][prediction]
    except:
        return "Nutral" , 0.5

In [14]:
sentence = "❤🖤❤"
Predict1(sentence)

('Positive', 88.84322)

In [15]:
# Prediction only

def Predict2(sentence):
    try:
        sentence = sentence[:1600]
        sentence = preprocess_text(sentence)
        labels = ["Positive", 'Negative', "Nutral"]
        inputs = tokenizer(sentence, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        logits = emoje_score(sentence, logits)
        prediction = int(torch.argmax(logits, dim=-1))
        logits_np = logits.cpu().detach().numpy()
        prob = (np.exp(logits_np) / np.sum(np.exp(logits_np))) * 100
        return labels[prediction]
    except:
        return "Nutral" 

In [16]:
sentence = "❤🖤❤"
Predict2(sentence)

'Positive'

## Prepare emojis data

In [ ]:
df = pd.read_csv("/content/Emoji_Sentiment_Data_v1.0.csv" ,encoding='utf-8')
df

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons
...,...,...,...,...,...,...,...,...,...
964,➛,0x279b,1,0.011628,0,1,0,DRAFTING POINT RIGHTWARDS ARROW,Dingbats
965,♝,0x265d,1,0.280000,0,1,0,BLACK CHESS BISHOP,Miscellaneous Symbols
966,❋,0x274b,1,0.888889,0,1,0,HEAVY EIGHT TEARDROP-SPOKED PROPELLER ASTERISK,Dingbats
967,✆,0x2706,1,0.557252,0,1,0,TELEPHONE LOCATION SIGN,Dingbats


In [ ]:
data = {'positive': [], 'negative': []}

def check_emoji(x):
    if x['Positive'] > x['Negative']:
        data['positive'].append(x['Emoji'])

    if x['Positive'] < x['Negative']:
        data['negative'].append(x['Emoji'])
    
    return data



In [ ]:
for i, row in df.T.iteritems():
    check_emoji(row)

In [ ]:

with open('/content/emojis.yml', 'w') as f:
    
    data = yaml.dump(data, f)
